In [1]:
import pandas as pd
from dask import dataframe as dd

In [2]:
df=dd.read_hdf('/data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8*.hdf','/data')

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0000.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0000.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0001.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0001.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0002.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0002.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0003.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0003.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0004.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0004.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0045.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0045.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0046.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0046.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0047.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0047.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0048.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0048.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0049.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0049.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0089.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0090.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0090.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0091.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0091.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0092.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0092.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0093.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0093.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0094.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0136.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0136.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0137.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0137.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0138.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0138.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0139.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0139.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0140.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0140.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0182.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0182.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0183.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0183.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0184.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0184.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0185.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0185.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0186.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0186.hdf in read-

...

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0545.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0545.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0546.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0546.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0547.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0547.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0548.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0548.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0549.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0549.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0588.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0588.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0589.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0589.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0590.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0590.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0591.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0591.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0592.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0592.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0632.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0633.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0633.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0634.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0634.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0635.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0635.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0636.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0636.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0637.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0678.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0679.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0679.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0680.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0680.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0681.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0681.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0682.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0682.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0683.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0726.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0726.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0727.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0727.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0728.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0728.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0729.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0729.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0730.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0730.hdf in read-

In [5]:
df.columns

Index([u'frame', u'face_id', u'timestamp', u'confidence', u'success',
       u'gaze_0_x', u'gaze_0_y', u'gaze_0_z', u'gaze_1_x', u'gaze_1_y',
       ...
       u'AU25_c', u'AU26_c', u'AU28_c', u'AU45_c', u'patient', u'session',
       u'vid', u'annotated', u'Happy_predicted', u'Happy_predicted_proba'],
      dtype='object', length=720)

In [12]:
imp_columns=['frame','vid','patient','session','timestamp','success','Happy_predicted', 'Happy_predicted_proba']
df=df[imp_columns].compute()

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0000.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0001.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0002.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0003.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0004.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0005.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0006.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0007.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0008.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0009.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0083.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0084.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0085.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0086.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0087.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0088.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0089.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0090.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0091.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0092.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0165.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0166.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0167.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0168.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0169.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0170.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0171.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0172.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0173.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0174.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0247.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0248.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0249.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0250.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0251.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0252.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0253.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0254.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0255.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0256.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0331.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0332.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0333.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0334.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0335.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0336.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0337.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0338.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0339.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0340.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0413.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0414.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0415.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0416.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0417.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0418.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0419.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0420.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0421.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0422.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0495.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0496.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0497.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0498.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0499.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0500.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0501.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0502.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0503.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0504.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0577.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0578.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0579.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0580.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0581.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0582.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0583.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0584.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0585.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0586.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0659.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0660.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0661.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0662.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0663.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0664.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0665.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0666.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0667.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0668.hdf in read-

Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0741.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0742.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0743.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0744.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0745.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0746.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0747.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0748.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0749.hdf in read-only mode
Opening /data/users/valentina/OpenFaceFeatures/all_aus_cleaned/cb46fd46_8_0750.hdf in read-

In [13]:
df

,frame,vid,patient,session,timestamp,success,Happy_predicted,Happy_predicted_proba
0,1.0,0000,cb46fd46,8,0.000,0.0,False,0.21
1,2.0,0000,cb46fd46,8,0.033,0.0,False,0.21
2,3.0,0000,cb46fd46,8,0.067,0.0,False,0.21
3,4.0,0000,cb46fd46,8,0.100,0.0,False,0.21
4,5.0,0000,cb46fd46,8,0.133,0.0,False,0.21
5,6.0,0000,cb46fd46,8,0.167,0.0,False,0.21
6,7.0,0000,cb46fd46,8,0.200,0.0,False,0.21
7,8.0,0000,cb46fd46,8,0.233,0.0,False,0.21
8,9.0,0000,cb46fd46,8,0.267,0.0,False,0.21
9,10.0,0000,cb46fd46,8,0.300,0.0,False,0.21
